## **RunnableConfig** 설정
- LangChain에서 런타임에 Runnable(실행 가능한 컴포넌트)의 동작을 세밀하게 제어하기 위한 설정 객체

**주요 특징**:
- 체인, 툴, 모델 등 다양한 Runnable에 전달되어 실행 시 동작을 조정
- 실행 중인 Runnable과 하위 호출들에 설정을 전달하는 컨텍스트 역할
- LangChain의 모든 Runnable은 이 설정을 받아 실행을 최적화하거나 동작 변경 가능

**주요 속성**:

1. **configurable**
   - **용도**: 런타임에 조정 가능한 속성 값 전달
   - **예시**: 모델의 온도, 세션 ID, 프롬프트 템플릿 등

2. **callbacks**
   - **용도**: 실행 과정에서 이벤트를 처리할 콜백 핸들러 지정
   - **예시**: 로깅, 모니터링, 이벤트 추적

3. **tags**
   - **용도**: 실행에 태그를 붙여 추적 및 필터링
   - **예시**: 실험 버전, 사용자 그룹, 요청 타입 등

4. **metadata**
   - **용도**: 실행 관련 추가 메타데이터 전달
   - **예시**: 요청 ID, 사용자 정보, 세션 데이터 등

- [LangChain RunnableConfig 공식 문서](https://python.langchain.com/docs/concepts/runnables/)

In [1]:
from dotenv import load_dotenv
load_dotenv()

# Langsmith tracing 여부를 확인 (true: langsmith 추적 활성화, false: langsmith 추적 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

true


### 1. 성능 모니터링 콜백 핸들러 구현

- LLM 호출의 성능을 실시간으로 모니터링하는 콜백 핸들러 구현
- `BaseCallbackHandler`를 상속하여 구현
- `on_llm_start`, `on_llm_end`, `on_llm_error` 메서드를 오버라이드
- 실행 시간, 토큰 사용량, 호출 횟수 등의 성능 지표 수집


In [2]:
import time
import logging
from datetime import datetime
from typing import Dict, List, Any, Optional
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.outputs import LLMResult

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

class PerformanceMonitoringCallback(BaseCallbackHandler):
    """LLM 호출 성능을 모니터링하는 콜백 핸들러"""
    
    def __init__(self):
        self.start_time: Optional[float] = None       # LLM 호출 시작 시간
        self.token_usage: Dict[str, Any] = {}         # 토큰 사용량 정보
        self.call_count: int = 0                      # LLM 호출 횟수
        
    def on_llm_start(
        self, 
        serialized: Dict[str, Any], 
        prompts: List[str], 
        **kwargs: Any
    ) -> None:
        """LLM 호출이 시작될 때 호출"""
        self.start_time = time.time()
        self.call_count += 1
        print(f"🚀 LLM 호출 #{self.call_count} 시작 - {datetime.now().strftime('%H:%M:%S')}")
        
        # 첫 번째 프롬프트의 길이 확인
        if prompts:
            print(f"📝 프롬프트 길이: {len(prompts[0])} 문자")
        
    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        """LLM 호출이 완료될 때 호출"""
        if self.start_time:
            duration = time.time() - self.start_time
            print(f"✅ LLM 호출 완료 - 소요시간: {duration:.2f}초")
            
            # 토큰 사용량 추적
            if response.generations:
                generation = response.generations[0][0]
            
                usage = response.llm_output.get('token_usage', {})
                if usage:
                    print(f"🔢 토큰 사용량: {usage}")
                    self.token_usage = usage
                        
                # 응답 길이 체크
                if hasattr(generation, 'text'):
                    response_text = generation.text
                    print(f"📊 응답 길이: {len(response_text)} 문자")
        
    def on_llm_error(self, error: Exception, **kwargs: Any) -> None:
        """LLM 호출에서 오류가 발생할 때 호출"""
        print(f"❌ LLM 호출 오류: {str(error)}")
        
    def get_statistics(self) -> Dict[str, Any]:
        """현재까지의 통계 정보를 반환"""
        return {
            "total_calls": self.call_count,
            "last_token_usage": self.token_usage
        }

### 2. 실시간 알림 콜백 핸들러 구현

- 특정 조건(비용 임계값, 응답 시간 등)에서 알림을 보내는 콜백 핸들러를 구현 (임계값 기반 알림 시스템 구현)
- 비용, 응답 시간, 프롬프트 길이 등 다양한 조건 모니터링
- 실제 프로덕션 환경에서는 외부 알림 서비스 연동 가능

In [3]:
class AlertCallback(BaseCallbackHandler):
    """특정 조건에서 알림을 보내는 콜백 핸들러"""
    
    def __init__(
        self, 
        cost_threshold: float = 1.0,            # 비용 임계값 (달러 단위)
        response_time_threshold: float = 10.0,  # 응답 시간 임계값 (초 단위)
        token_threshold: int = 4000             # 긴 프롬프트 토큰 임계값
    ):
        self.cost_threshold = cost_threshold
        self.response_time_threshold = response_time_threshold
        self.token_threshold = token_threshold
        self.start_time: Optional[float] = None    # LLM 호출 시작 시간
        self.cumulative_cost: float = 0.0          # 누적 비용 추적
        
    def on_llm_start(
        self, 
        serialized: Dict[str, Any], 
        prompts: List[str], 
        **kwargs: Any
    ) -> None:
        """LLM 호출이 시작될 때 호출"""
        self.start_time = time.time()
        
        # 긴 프롬프트 경고
        if prompts and len(prompts[0]) > self.token_threshold:
            self._send_alert(f"⚠️ 긴 프롬프트 감지: {len(prompts[0])} 문자")
    
    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        """LLM 호출이 완료될 때 호출"""
        # 응답 시간 체크
        if self.start_time:
            duration = time.time() - self.start_time
            if duration > self.response_time_threshold:  # 임계값 초과 시 알림
                self._send_alert(f"🐌 느린 응답: {duration:.2f}초")
        
        # 비용 체크
        if response.generations:
            generation = response.generations[0][0]
            usage = response.llm_output.get('token_usage', {})
            
            if usage:
                # 간단한 비용 계산 (실제로는 모델별 가격 적용 필요)
                total_tokens = usage.get('total_tokens', 0)
                if total_tokens == 0:
                    # input_tokens와 output_tokens로 계산
                    total_tokens = usage.get('input_tokens', 0) + usage.get('output_tokens', 0)
                
                estimated_cost = (total_tokens / 1000) * 0.002
                self.cumulative_cost += estimated_cost
                
                if self.cumulative_cost > self.cost_threshold:
                    self._send_alert(f"💸 비용 임계값 초과: ${self.cumulative_cost:.4f}")
    
    def on_llm_error(self, error: Exception, **kwargs: Any) -> None:
        """LLM 호출에서 오류가 발생할 때 호출"""
        self._send_alert(f"🚨 LLM 오류 발생: {str(error)}")
    
    def _send_alert(self, message: str) -> None:
        """실제 환경에서는 Slack, Discord, 이메일 등으로 알림을 보냄"""
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        alert_message = f"[ALERT] {timestamp} - {message}"
        print("🔔 알림 전송:", alert_message)
        
        # 로깅 시스템에 기록
        logging.warning(alert_message)
        
        # 실제 구현에서는 아래와 같은 방식으로 외부 서비스에 알림 전송
        # self._send_slack_notification(message)
        # self._send_email_notification(message)
    
    def reset_cost_tracking(self) -> None:
        """누적 비용 추적을 리셋"""
        self.cumulative_cost = 0.0

### 3. RunnableConfig 기본 사용

- RunnableConfig를 사용하여 체인 실행을 제어하고 모니터링
- `configurable_fields`를 통한 런타임 설정 가능한 모델 생성
- RunnableConfig의 4가지 주요 속성 활용
- 콜백 핸들러를 통한 실시간 성능 모니터링

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import ConfigurableField

# 설정 가능한 모델 생성 - temperature를 런타임에 변경 가능하게 설정
model = ChatOpenAI(
    model="gpt-4.1-mini",  
    temperature=0.3,      # 기본값
    top_p=0.95
).configurable_fields(    # temperature 필드를 런타임에 변경 가능하게 설정
    temperature=ConfigurableField(
        id="temperature",
        name="Model Temperature",
        description="모델의 창의성을 조절하는 온도 매개변수"
    )
)

prompt = PromptTemplate.from_template("'{text}'를 영어로 번역해주세요. 번역된 문장만을 출력해주세요.")
output_parser = StrOutputParser()
translation_chain = prompt | model | output_parser

# 콜백 핸들러 생성
performance_handler = PerformanceMonitoringCallback()

# 체인 실행
result = translation_chain.invoke(
    {
        "text": "안녕하세요, 오늘 날씨는 어떠신가요?"
    },
    config={
        "configurable": {"temperature": 0.7, "session_id": "user123"},   # 런타임에 temperature를 변경 가능
        "callbacks": [performance_handler],
        "tags": ["experiment_v1", "production"],
        "metadata": {"request_id": "req_001", "user_id": "user123"},
    }
)

h:\miniconda3\envs\modu2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 LLM 호출 #1 시작 - 17:34:47
📝 프롬프트 길이: 59 문자


2025-09-27 17:34:48,155 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


✅ LLM 호출 완료 - 소요시간: 0.62초
🔢 토큰 사용량: {'completion_tokens': 8, 'prompt_tokens': 38, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}
📊 응답 길이: 32 문자


In [5]:
print(f"번역 결과: {result}")

번역 결과: Hello, how is the weather today?


### 4. 알림 조건 테스트

- AlertCallback의 다양한 알림 조건을 테스트

In [6]:
# 조건을 완화해서 테스트
alert_handler_test = AlertCallback(
    cost_threshold=0.0,  # $0.0 임계값 -> 모든 호출에 대해 알림
    response_time_threshold=0.1,  # 0.1초 임계값 -> 대부분의 호출에서 알림
    token_threshold=10  # 10 문자 임계값 -> 짧은 프롬프트도 알림
)

# 테스트 실행
test_result = translation_chain.invoke(
    {
        "text": "Hello World!"  # 짧은 텍스트로 테스트
    },
    config={
        "configurable": {"temperature": 0.1},
        "callbacks": [performance_handler, alert_handler_test],  # 성능 모니터링과 알림 콜백 핸들러 모두 사용
        "tags": ["test", "alert_validation"],
        "metadata": {"request_id": "test_001", "test_type": "alert_threshold"},
    }
)

2025-09-27 17:34:48,192 - root - WARNING - [ALERT] 2025-09-27 17:34:48 - ⚠️ 긴 프롬프트 감지: 51 문자


🚀 LLM 호출 #2 시작 - 17:34:48
📝 프롬프트 길이: 51 문자
🔔 알림 전송: [ALERT] 2025-09-27 17:34:48 - ⚠️ 긴 프롬프트 감지: 51 문자


2025-09-27 17:34:48,703 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-27 17:34:48,712 - root - WARNING - [ALERT] 2025-09-27 17:34:48 - 🐌 느린 응답: 0.52초
2025-09-27 17:34:48,713 - root - WARNING - [ALERT] 2025-09-27 17:34:48 - 💸 비용 임계값 초과: $0.0001


✅ LLM 호출 완료 - 소요시간: 0.52초
🔢 토큰 사용량: {'completion_tokens': 3, 'prompt_tokens': 28, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}
📊 응답 길이: 12 문자
🔔 알림 전송: [ALERT] 2025-09-27 17:34:48 - 🐌 느린 응답: 0.52초
🔔 알림 전송: [ALERT] 2025-09-27 17:34:48 - 💸 비용 임계값 초과: $0.0001


In [7]:
print(f"테스트 결과: {test_result}")

테스트 결과: Hello World!


In [8]:
# 통계 정보 확인
stats = performance_handler.get_statistics()
print(f"총 호출 횟수: {stats['total_calls']}")
print(f"마지막 토큰 사용량: {stats['last_token_usage']}")

총 호출 횟수: 2
마지막 토큰 사용량: {'completion_tokens': 3, 'prompt_tokens': 28, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}


**[참고]** [LangSmith Alert 설정](https://docs.smith.langchain.com/observability/how_to_guides/alerts)

### 5. 기타 활용법

`(1) 동적 모델 선택`

In [9]:
# model 변수의 모델 이름 출력
print("현재 모델 이름:", model.model_name)

현재 모델 이름: gpt-4.1-mini


In [10]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# 설정 가능한 모델 생성 (여러 모델을 선택할 수 있도록 설정)
model_alternatives = model.configurable_alternatives(
    ConfigurableField(id="model_name"),
    default_key="gpt4_mini",  # model 변수의 기본 모델을 식별하는 키
    openai_gpt4=ChatOpenAI(model="gpt-4.1"),  # OpenAI GPT-4 모델
    google_gemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash"),  # Google Gemini 모델
)

# chain 설정 
translation_chain = prompt | model_alternatives | output_parser

# 기본 모델로 번역 실행 (default_key: gpt4_mini)
result = translation_chain.invoke(
    {"text": "안녕하세요, 오늘 날씨는 어떠신가요?"}
)

print("번역 결과:", result)

2025-09-27 17:34:50,485 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


번역 결과: Hello, how is the weather today?


In [11]:
# 런타임에 모델 변경 (google_gemini 키 사용 -> gemini-2.0-flash 모델 사용)
result = translation_chain.invoke(
    {"text": "안녕하세요, 오늘 날씨는 어떠신가요?"},
    config={"configurable": {"model_name": "google_gemini"}} 
)

print("번역 결과:", result)

번역 결과: Hello, how is the weather today?


In [12]:
# 런타임에 모델 변경  (openai_gpt4 키 사용 -> gpt-4.1 모델 사용)
result = translation_chain.invoke(
    {"text": "안녕하세요, 오늘 날씨는 어떠신가요?"},
    config={"configurable": {"model_name": "openai_gpt4"}}
)

print(result)

2025-09-27 17:34:51,943 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Hello, how is the weather today?


`(2) 동적 프롬프트 선택`

In [13]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import ConfigurableField
from langchain_core.prompts import ChatPromptTemplate

# 여러 모델을 alternatives로 설정
model = ChatOpenAI(model="gpt-4.1-mini").configurable_alternatives(
    ConfigurableField(id="model_provider"),
    default_key="gpt4_mini",
    openai_gpt4=ChatOpenAI(model="gpt-4.1"),
    google_gemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash")
).configurable_fields(
    temperature=ConfigurableField(
        id="temperature",
        name="Temperature",
        description="Model temperature for response randomness"
    ),
    max_tokens=ConfigurableField(
        id="max_tokens",
        name="Max Tokens", 
        description="Maximum tokens to generate"
    )
)

# 기본 프롬프트
default_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 도움이 되는 AI 어시스턴트입니다. 모든 질문에 최선을 다해 답변하세요."),
    ("human", "{user_input}")
])

# 창의적 프롬프트
creative_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 창의적인 AI 어시스턴트입니다. 상상력을 발휘하여 답변하세요."),
    ("human", "{user_input}")
])

# 분석적 프롬프트
analytical_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 분석적인 AI 어시스턴트입니다. 논리적으로 사고하여 답변하세요."),
    ("human", "{user_input}")
])

# 프롬프트를 configurable alternatives로 설정
prompt = default_prompt.configurable_alternatives(
    ConfigurableField(id="prompt_type"),
    default_key="default",
    creative=creative_prompt,
    analytical=analytical_prompt
)

# chain 설정
chain = prompt | model

In [14]:
# 기본 프롬프트로 실행
basic_config = {
    "configurable": {
        "model_provider": "gpt4_mini",
        "temperature": 0.5,
        "max_tokens": 500,
        "prompt_type": "default"
    }
}

result1 = chain.invoke(
    {"user_input": "인공지능에 대해 알려주세요"}, 
    config=basic_config
)

print(f"기본 프롬프트 결과: {result1.content}")

2025-09-27 17:34:58,367 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


기본 프롬프트 결과: 인공지능(人工知能, Artificial Intelligence, AI)은 인간의 지능을 기계나 컴퓨터 시스템에 구현하는 기술과 학문 분야를 말합니다. AI는 데이터를 분석하고 학습하며, 문제를 해결하거나 의사결정을 지원하는 능력을 갖추도록 설계됩니다.

### 인공지능의 주요 개념
1. **기계 학습(Machine Learning)**  
   컴퓨터가 명시적인 프로그래밍 없이 데이터를 통해 스스로 학습하고 성능을 향상시키는 기술입니다. 대표적인 예로는 지도학습, 비지도학습, 강화학습 등이 있습니다.

2. **딥러닝(Deep Learning)**  
   인공신경망을 기반으로 한 기계학습의 한 분야로, 다층 신경망을 통해 복잡한 데이터의 패턴을 학습합니다. 이미지 인식, 음성 인식, 자연어 처리 등에 널리 사용됩니다.

3. **자연어 처리(Natural Language Processing, NLP)**  
   컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다. 번역, 챗봇, 음성 인식 등이 포함됩니다.

4. **컴퓨터 비전(Computer Vision)**  
   이미지나 동영상에서 유용한 정보를 추출하는 기술로, 얼굴 인식, 객체 탐지 등에 활용됩니다.

### 인공지능의 활용 분야
- **의료:** 질병 진단, 신약 개발, 의료 영상 분석
- **자동차:** 자율주행차, 교통 관리
- **금융:** 신용 평가, 투자 분석, 사기 탐지
- **고객 서비스:** 챗봇, 음성 비서
- **제조:** 스마트 팩토리, 품질 검사

### 인공지능의 장점과 한계
- **장점:** 대량 데이터 처리, 인간이 하기 어려운 복잡한 문제 해결, 업무 자동화
- **한계:** 데이터 편향 문제, 윤리적 이슈, 완전한 인간 지능 대체 불가

필요하시면 특정 분야나 기술에 대해 더 자세히 설명해 드릴 수 있습니다.


In [15]:
# 창의적 프롬프트로 실행
creative_config = {
    "configurable": {
        "model_provider": "google_gemini",
        "temperature": 0.9,
        "max_tokens": 700,
        "prompt_type": "creative"
    }
}

result2 = chain.invoke(
    {"user_input": "인공지능에 대해 알려주세요"}, 
    config=creative_config
)
print(f"창의적 프롬프트 결과: {result2.content}")

창의적 프롬프트 결과: 자, 인공지능에 대한 이야기는 놀라운 모험과 같습니다! 마치 우리가 상상력으로 가득 찬 로봇 친구를 만드는 것과 같아요.

인공지능(AI)은 컴퓨터가 마치 사람처럼 생각하고 배울 수 있도록 만드는 기술입니다. AI는 거대한 두뇌와 같아서, 많은 정보를 빠르게 처리하고 새로운 것을 배울 수 있습니다.

AI는 우리 생활 곳곳에 숨어 있어요. 예를 들어, 스마트폰에서 "안녕, AI"라고 말하면 AI가 우리의 말을 듣고 대답해 줍니다. 또, 영화를 추천해 주는 AI는 우리가 좋아할 만한 영화를 척척 골라주죠. 운전하는 로봇 자동차도 있고, 그림을 그리는 AI 화가도 있답니다!

AI는 배우는 것을 아주 좋아합니다. 마치 어린 아이가 세상을 탐험하며 배우듯이, AI는 수많은 데이터를 분석하고 패턴을 찾아냅니다. 그리고 그 패턴을 바탕으로 미래를 예측하거나 새로운 것을 만들어내기도 합니다.

하지만 AI는 아직 완벽하지 않아요. 때로는 엉뚱한 대답을 하거나 실수를 하기도 합니다. 그래서 우리는 AI를 계속 가르치고 훈련시켜야 합니다.

AI의 미래는 정말 흥미진진합니다. AI는 우리의 삶을 더욱 편리하고 풍요롭게 만들어 줄 수 있습니다. 하지만 동시에 AI가 우리에게 어떤 영향을 미칠지 고민하고, AI를 올바르게 사용하는 방법을 배워야 합니다.

자, 이제 AI의 세계로 함께 떠나볼까요? 어떤 놀라운 일들이 우리를 기다리고 있을지 기대되지 않나요!


---

## **LangChain Fallback 처리**
- **Fallback**은 비상 상황에서 사용할 수 있는 대안적 계획을 의미
- LangChain에서는 주요 실행 경로가 실패했을 때 자동으로 대체 경로를 실행하는 메커니즘을 제공

**필요성**:

1. **API 안정성 문제**
    - 요금 제한(Rate Limiting)
    - 서버 다운타임
    - 네트워크 오류
    - API 키 할당량 초과

2. **비용 최적화**
    - 저렴한 모델을 먼저 시도
    - 실패 시에만 비싼 모델 사용

3. **성능 최적화**
   - 빠른 모델을 우선 사용
    - 복잡한 작업에만 고성능 모델 사용

4. **컨텍스트 길이 제한**
   - 짧은 컨텍스트 모델을 먼저 시도
   - 토큰 초과 시 긴 컨텍스트 모델 사용

- [LangChain Fallbacks 가이드](https://python.langchain.com/docs/how_to/fallbacks/)


### 1. API 오류에 대한 Fallback 처리

- OpenAI API 오류 발생 시 Google Gemini로 자동 전환하는 시스템 구현
- `max_retries=0` 설정으로 즉시 fallback 전환
- 여러 단계의 fallback 체인 구성 가능

`(1) 기본 Fallback 구현`

In [16]:
# 기본 모델과 fallback 모델 설정
# max_retries=0으로 설정하여 즉시 fallback으로 전환
primary_model = ChatOpenAI(
    model="gpt-4.1-mini", 
    max_retries=0,  # 재시도 없이 바로 fallback으로 전환
    temperature=0.7
)

fallback_model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", 
    temperature=0.7
)

# Fallback 체인 생성
llm_with_fallback = primary_model.with_fallbacks([fallback_model])

`(2) 테스트용 오류 시뮬레이션`

In [17]:
from unittest.mock import patch
import httpx
from openai import RateLimitError

# 테스트용 오류 객체 생성
def create_mock_error():
    request = httpx.Request("GET", "/")
    response = httpx.Response(429, request=request)  # 429 = 너무 많은 요청
    return RateLimitError("Rate limit exceeded", response=response, body="")

# 오류 시뮬레이션 테스트
error = create_mock_error()

print("=== Primary 모델만 사용 (오류 발생) ===")
# `patch`를 사용하여 OpenAI API 호출을 Mocking (side_effect를 error로 설정)
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        result = primary_model.invoke("달과 지구 사이의 거리는?")
        print(f"결과: {result.content}")
    except RateLimitError as e:
        print(f"오류 발생: {e}")

print("\n=== Fallback이 적용된 모델 사용 ===")
# `patch`를 사용하여 OpenAI API 호출을 Mocking (side_effect를 error로 설정)
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        result = llm_with_fallback.invoke("달과 지구 사이의 거리는?")
        print(f"결과: {result.content[:200]}...")
        print("✅ Fallback 모델로 자동 전환 성공!")
    except Exception as e:
        print(f"오류 발생: {e}")

=== Primary 모델만 사용 (오류 발생) ===
오류 발생: Rate limit exceeded

=== Fallback이 적용된 모델 사용 ===
결과: 달과 지구 사이의 거리는 일정하지 않고 타원 궤도를 돌기 때문에 변합니다. 

*   **평균 거리:** 약 384,400km (238,900마일)

*   **최근접점 (근지점):** 약 363,104km (225,623마일)

*   **최원점 (원지점):** 약 405,696km (252,088마일)...
✅ Fallback 모델로 자동 전환 성공!


`(3) Fallback 체인 예시`

In [18]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 체인 생성 함수
def create_fallback_chat_chain():
    """Fallback 채팅 체인 생성"""

    # 프롬프트 템플릿
    prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 친절하고 도움이 되는 AI 어시스턴트입니다. 항상 예의바르게 답변해주세요."),
        ("human", "{user_input}")
    ])
    
    # 여러 단계의 fallback 설정
    primary = ChatOpenAI(model="gpt-4.1-mini", max_retries=0, temperature=0.7)
    fallback1 = ChatGoogleGenerativeAI(model="gemini-2.0-flash", max_retries=0, temperature=0.7)
    fallback2 = ChatOllama(model="qwen3:4b", temperature=0.7)

    # 3단계 fallback 체인
    robust_llm = primary.with_fallbacks([fallback1, fallback2])
    
    # 완전한 체인 구성
    chain = prompt | robust_llm | StrOutputParser()
    
    return chain

# 견고한 체인 테스트
robust_chain = create_fallback_chat_chain()

# 정상 작동 테스트
print("=== 정상 작동 테스트 ===")
try:
    response = robust_chain.invoke({"user_input": "LangChain의 장점을 3가지만 설명해주세요."})
    print(f"응답: {response}")
except Exception as e:
    print(f"오류: {e}")

=== 정상 작동 테스트 ===


2025-09-27 17:35:07,821 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


응답: 물론입니다! LangChain의 주요 장점 세 가지를 말씀드리자면:

1. **유연한 체인 구성**  
   LangChain은 다양한 작업을 연결하여 복잡한 워크플로우를 쉽게 구성할 수 있게 해줍니다. 예를 들어, 텍스트 생성, 검색, 데이터 추출 등의 여러 단계를 하나의 체인으로 만들어 자동화할 수 있습니다.

2. **다양한 데이터 소스 통합 지원**  
   여러 데이터베이스, API, 문서, 검색 엔진 등 다양한 외부 소스와 쉽게 연동할 수 있어, 풍부한 정보를 바탕으로 한 지능형 애플리케이션 개발이 용이합니다.

3. **언어 모델과의 원활한 통합**  
   OpenAI, Hugging Face 등 다양한 대형 언어 모델(LLM)과 손쉽게 통합할 수 있어서 최신 AI 기술을 빠르게 적용하고 실험할 수 있습니다.

필요하시면 더 자세한 설명도 드릴 수 있습니다!


### 2. 모델별 최적화 적용

- 각기 다른 모델에 최적화된 프롬프트를 사용하는 fallback 시스템을 구현
- 각 모델의 특성에 맞는 프롬프트 최적화
- 성능-비용 균형을 고려한 다단계 fallback
- 프로덕션 환경에서의 안정성 확보

In [19]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser

def create_production_qa_system():
    """모델별 최적화 Q&A 시스템"""
    
    # 고성능 모델용 프롬프트 (gpt-4.1 적용)
    premium_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 전문적인 분석가입니다. 
        다음 지침을 따라주세요:
        1. 질문을 깊이 분석하세요
        2. 다각도에서 접근하세요  
        3. 구체적인 예시를 포함하세요
        4. 결론을 명확히 제시하세요"""),
        ("human", "{question}")
    ])
    
    # 기본 모델용 프롬프트 (gpt-4.1-mini 적용)
    standard_prompt = ChatPromptTemplate.from_messages([
        ("system", "간결하고 정확한 답변을 제공해주세요."),
        ("human", "{question}")
    ])
    
    # 경량 모델용 프롬프트 (qwen3:4b 적용)
    budget_prompt = PromptTemplate.from_template(
        "질문: {question}\n\n핵심 답변:"
    )
    
    # 모델 체인 구성
    premium_chain = premium_prompt | ChatOpenAI(model="gpt-4.1", temperature=0.3, max_retries=0)
    standard_chain = standard_prompt | ChatOpenAI(model="gpt-4.1-mini", temperature=0.3, max_retries=0)  
    budget_chain = budget_prompt | ChatOllama(model="qwen3:4b", temperature=0.3)
    
    # 3단계 fallback 시스템
    qa_system = premium_chain.with_fallbacks([
        standard_chain, 
        budget_chain
    ]) | StrOutputParser()
    
    return qa_system

# 모델별 최적화 Q&A 시스템 테스트
production_qa = create_production_qa_system()

test_questions = [
    "인공지능의 미래 전망은 어떻게 될까요?",
    "기업에서 AI를 도입할 때 고려해야 할 요소들은?",
    "머신러닝과 딥러닝의 차이점은?"
]

for i, question in enumerate(test_questions, 1):
    print(f"\n=== 질문 {i}: {question} ===")
    try:
        answer = production_qa.invoke({"question": question})
        print(f"답변: {answer[:300]}...")
    except Exception as e:
        print(f"오류: {e}")


=== 질문 1: 인공지능의 미래 전망은 어떻게 될까요? ===


2025-09-27 17:35:19,814 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


답변: 질문 분석  
“인공지능의 미래 전망”은 매우 포괄적인 주제입니다. 여기에는 기술 발전, 사회적 영향, 경제적 변화, 윤리적 쟁점, 산업별 적용 등 다양한 측면이 포함됩니다. 따라서 이 질문에 대한 답변은 여러 각도에서 접근해야 하며, 단순히 기술적 진보만이 아니라 사회 전반에 미치는 영향을 함께 고려해야 합니다.

다각도 접근  
1. 기술적 발전  
- 딥러닝, 강화학습, 생성형 AI(예: ChatGPT, Midjourney 등) 등 인공지능의 핵심 기술은 빠르게 발전하고 있습니다.  
- 앞으로는 멀티모달 AI(텍스트, 이미지...

=== 질문 2: 기업에서 AI를 도입할 때 고려해야 할 요소들은? ===


2025-09-27 17:35:30,361 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


답변: 질문 분석  
“기업에서 AI를 도입할 때 고려해야 할 요소들은?”이라는 질문은 단순히 기술적 도입만이 아니라, 기업의 전략, 조직, 비용, 윤리 등 다양한 측면을 포괄합니다. AI 도입은 기업의 경쟁력 강화, 업무 효율화, 새로운 비즈니스 모델 창출 등 긍정적 효과가 있지만, 실패 시 비용 손실, 데이터 유출, 조직 저항 등 부정적 결과도 초래할 수 있습니다. 따라서 다각도에서 접근이 필요합니다.

다각도 접근  
1. 전략적 측면  
   - 비즈니스 목표와의 정합성: AI 도입이 기업의 장기적 목표와 어떻게 연결되는지 분석해야...

=== 질문 3: 머신러닝과 딥러닝의 차이점은? ===


2025-09-27 17:35:42,939 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


답변: 질문 분석  
"머신러닝과 딥러닝의 차이점"은 인공지능(AI) 분야에서 매우 자주 등장하는 질문입니다. 두 용어 모두 데이터 기반 학습을 통해 문제를 해결한다는 공통점이 있지만, 그 원리와 적용 방식, 성능, 필요 조건 등에서 중요한 차이점이 존재합니다.

다각도 접근  
1. 개념적 차이  
- 머신러닝(Machine Learning):  
  머신러닝은 데이터로부터 패턴을 학습하여 예측이나 분류 등의 작업을 수행하는 알고리즘의 집합입니다. 대표적으로 의사결정트리, 서포트 벡터 머신(SVM), 랜덤 포레스트, K-최근접 이웃(KN...
